In [1]:
import pickle
def reload_vars(filename):
    this_var = None
    with open(filename, 'rb') as file:
        this_var = pickle.load(file)
        
    return this_var

def pkl_vars(varname, filename):
    with open(filename, 'wb') as file:
        pickle.dump(varname, file)

In [2]:
from ast import literal_eval
from seqeval import scheme

import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import pipeline, AutoModelForTokenClassification, AutoTokenizer, get_cosine_schedule_with_warmup
from seqeval.metrics import classification_report
import math
import os
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'
print(device)

2025-05-03 20:44:10.607900: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-05-03 20:44:10.766462: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-05-03 20:44:11.297730: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2025-05-03 20:44:11.297819: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinf

cuda


In [21]:
TRAIN_BATCH_SIZE = 8
VALID_BATCH_SIZE = 32
EPOCHS = 15#3#20
LEARNING_RATE = 5e-5 #1e-05
MAX_GRAD_NORM = 10
MAX_LEN = 256
model_name = "dbmdz/bert-base-german-europeana-cased"

In [4]:
import transformers, logging
transformers.logging.set_verbosity_error()

LOG_FORMAT = '%(asctime)s : %(filename)s : %(funcName)s : %(levelname)s : %(message)s'
logging.basicConfig(filename='FT_NER_xlm-roberta-base.log', level=logging.INFO, format=LOG_FORMAT)
logger = logging.getLogger("FT_NER_xlm-roberta-base")

In [20]:
!nvidia-smi

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Sat May  3 22:42:00 2025       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.39.01    Driver Version: 510.39.01    CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100S-PCI...  On   | 00000000:21:00.0 Off |                    0 |
| N/A   36C    P0    35W / 250W |  20032MiB / 32768MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  Tesla V100S-PCI...  On   | 00000000:81:00.0 Off |                    0 |
| N/A   

# Preprocessing the dataset

In [14]:
from collections import Counter

In [15]:
import json

In [16]:
f = open("data/REF.json")
REF_JSON = json.load(f)

In [17]:
RAW_DATA = []
ALL_TAG_COUNTER = Counter()


with open('data/ddb-ner-dataset.conll', encoding="utf-8") as f:
    sent_id = 0
    tokens = []
    ner_tags = []
    for line in f:
        if line == "" or line == "\n":
            if tokens:
                RAW_DATA.append({
                    "sentence": REF_JSON[sent_id]["title"],
                    "df_idx": REF_JSON[sent_id]["df_idx"],
                    "tokens": tokens,
                    "ner_tags": ner_tags,
                })
                sent_id += 1
                tokens = []
                ner_tags = []
        else:
            # SROIE2019 tokens are space separated
            splits = line.split(" ")
            tokens.append(splits[0])
            ner_tags.append(splits[3].strip())
            ALL_TAG_COUNTER.update({splits[3].strip(): 1})

    # last example
    RAW_DATA.append({
        "sentence": REF_JSON[sent_id]["title"],
        "df_idx": REF_JSON[sent_id]["df_idx"],
        "tokens": tokens,
        "ner_tags": ner_tags,

    })

pkl_vars(RAW_DATA, 'data/vars/RAW_DATA_2025.05.01.pkl')
df = pd.DataFrame(RAW_DATA)

In [78]:
ne_labels = sorted(ALL_TAG_COUNTER.keys())
ne_labels

['',
 'B-DATE-CREATION',
 'B-DATE-PUB',
 'B-DATE-SUBJ',
 'B-GPE-AUT',
 'B-GPE-CREATION',
 'B-GPE-DES',
 'B-GPE-PUB',
 'B-GPE-SUBJ',
 'B-LITWORK',
 'B-ORG-CREATION',
 'B-ORG-DES',
 'B-ORG-PUB',
 'B-ORG-SUBJ',
 'B-PERSON-AUT',
 'B-PERSON-CON',
 'B-PERSON-EDT',
 'B-PERSON-PUB',
 'B-PERSON-SUBJ',
 'B-YEAR-CREATION',
 'B-YEAR-PUB',
 'B-YEAR-SUBJ',
 'I-DATE-CREATION',
 'I-DATE-PUB',
 'I-DATE-SUBJ',
 'I-GPE',
 'I-GPE-CREATION',
 'I-GPE-DES',
 'I-GPE-PUB',
 'I-GPE-SUBJ',
 'I-I-GPE-DES',
 'I-LITWORK',
 'I-ORG',
 'I-ORG-CREATION',
 'I-ORG-DES',
 'I-ORG-PUB',
 'I-ORG-SUBJ',
 'I-PERSON-AUT',
 'I-PERSON-CON',
 'I-PERSON-EDT',
 'I-PERSON-PUB',
 'I-PERSON-SUBJ',
 'I-YEAR-CREATION',
 'I_ORG-DES',
 'O']

# Preparing the dataset and dataloader

Now that our data is preprocessed, we can turn it into PyTorch tensors such that we can provide it to the model. Let's start by defining some key variables that will be used later on in the training/evaluation process:

In [8]:
def tokenize_and_preserve_labels(tokens, ner_tags, tokenizer):
    """
    Word piece tokenization makes it difficult to match word labels
    back up with individual word pieces. This function tokenizes each
    word one at a time so that it is easier to preserve the correct
    label for each subword. It is, of course, a bit slower in processing
    time, but it will help our model achieve higher accuracy.
    """

    tokenized_sentence = []
    labels = []

    for word, label in zip(tokens, ner_tags):

        # Tokenize the word and count # of subwords the word is broken into
        tokenized_word = tokenizer.tokenize(word)
        n_subwords = len(tokenized_word)

        # Add the tokenized word to the final tokenized word list
        tokenized_sentence.extend(tokenized_word)

        # Add the same label to the new list of labels `n_subwords` times
        labels.extend([label] * n_subwords)

    return tokenized_sentence, labels

label2id = reload_vars('data/vars/label2id')
id2label = reload_vars('data/vars/id2label')


class dataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_len):
        self.len = len(dataframe)
        self.data = dataframe
        self.tokenizer = tokenizer
        self.max_len = max_len
        
    def __getitem__(self, index):
        # step 1: tokenize (and adapt corresponding labels)
        words = self.data.tokens[index]  
        word_labels = self.data.ner_tags[index]  
        tokenized_sentence, labels = tokenize_and_preserve_labels(words, word_labels, self.tokenizer)
        
        # step 2: add special tokens (and corresponding labels)
        tokenized_sentence = ["[CLS] "] + tokenized_sentence + [" [SEP]"] # add special tokens of Roberta
        labels.insert(0, 'O') # add outside label for [CLS] token
        labels.append('O') # add outside label for [SEP] token

        # step 3: truncating/padding
        maxlen = self.max_len

        if (len(tokenized_sentence) > maxlen):
            # truncate
            tokenized_sentence = tokenized_sentence[:maxlen]
            labels = labels[:maxlen]
        else:
            # pad
            tokenized_sentence = tokenized_sentence + ['[pad]'for _ in range(maxlen - len(tokenized_sentence))]
            labels = labels + ['O' for _ in range(maxlen - len(labels))]

        # step 4: obtain the attention mask
        attn_mask = [1 if tok != '[pad]' else 0 for tok in tokenized_sentence] #modifié selon https://huggingface.co/docs/transformers/v4.21.1/en/model_doc/camembert
        
        # step 5: convert tokens to input ids
        ids = self.tokenizer.convert_tokens_to_ids(tokenized_sentence)
        label_ids = [label2id[label] for label in labels]
        # the following line is deprecated
        #label_ids = [label if label != 0 else -100 for label in label_ids]
        
        return {
              'ids': torch.tensor(ids, dtype=torch.long),
              'mask': torch.tensor(attn_mask, dtype=torch.long),
              #'token_type_ids': torch.tensor(token_ids, dtype=torch.long),
              'targets': torch.tensor(label_ids, dtype=torch.long)
        } 
    
    def __len__(self):
        return self.len


## tests and data split

In [75]:
RAW_DATA = reload_vars('data/vars/RAW_DATA_2025.05.01.pkl')
df = pd.DataFrame(RAW_DATA)

In [9]:
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [11]:
for token, tag in zip(RAW_DATA[0]["tokens"], RAW_DATA[0]["ner_tags"]):
    print('{0:15}  {1}'.format(token, tag))

Hochzeit-Schertz  O
/                O
In               O
Vergleichung     O
des              O
süssen           O
Ehstandes        O
mit              O
dem              O
Baw              O
des              O
lieben           O
Berg-Wercks      O
:                O
Bey              O
Des              O
...              O
Herrn            O
Jobst            B-PERSON-SUBJ
Sellenstedts     I-PERSON-SUBJ
/                O
Fürstl.          O
Braunschw.       B-GPE-DES
und              I-GPE-DES
Lünäburg         I-GPE-DES
.                O
wolbestelten     O
Zehnd            O
:                O
Gegenschreibers  O
auff             O
der              O
Fürstl.          B-GPE-SUBJ
Bergstadt        I-GPE-SUBJ
Claußthal        I-GPE-SUBJ
/                O
angetretenen     O
andern           O
Eh               O
/                O
Mit              O
der              O
...              O
Jungfer          O
Christinen       B-PERSON-SUBJ
/                O
Des              O
...              O


In [51]:
tokenized_sentence, tags = tokenize_and_preserve_labels(df["tokens"][0], df["ner_tags"][0],tokenizer )
for token, tag in zip(tokenized_sentence, tags):
    print('{0:15}  {1}'.format(token, tag))

▁Hochzeit        O
-                O
Sch              O
ert              O
z                O
▁/               O
▁In              O
▁Vergleich       O
ung              O
▁des             O
▁süs             O
sen              O
▁Eh              O
stand            O
es               O
▁mit             O
▁dem             O
▁Ba              O
w                O
▁des             O
▁lieben          O
▁Berg            O
-                O
Wer              O
cks              O
▁:               O
▁Bey             O
▁Des             O
▁...             O
▁Herr            O
n                O
▁Jobs            B-PERSON-SUBJ
t                B-PERSON-SUBJ
▁Selle           I-PERSON-SUBJ
n                I-PERSON-SUBJ
stedt            I-PERSON-SUBJ
s                I-PERSON-SUBJ
▁/               O
▁Für             O
st               O
l                O
.                O
▁Braun           B-GPE-DES
sch              B-GPE-DES
w                B-GPE-DES
.                B-GPE-DES
▁und             I-GPE

In [79]:
label2id = { label : ne_labels.index(label) for label in ne_labels}

id2label = { ne_labels.index(label) : label for label in ne_labels}

label2id


{'': 0,
 'B-DATE-CREATION': 1,
 'B-DATE-PUB': 2,
 'B-DATE-SUBJ': 3,
 'B-GPE-AUT': 4,
 'B-GPE-CREATION': 5,
 'B-GPE-DES': 6,
 'B-GPE-PUB': 7,
 'B-GPE-SUBJ': 8,
 'B-LITWORK': 9,
 'B-ORG-CREATION': 10,
 'B-ORG-DES': 11,
 'B-ORG-PUB': 12,
 'B-ORG-SUBJ': 13,
 'B-PERSON-AUT': 14,
 'B-PERSON-CON': 15,
 'B-PERSON-EDT': 16,
 'B-PERSON-PUB': 17,
 'B-PERSON-SUBJ': 18,
 'B-YEAR-CREATION': 19,
 'B-YEAR-PUB': 20,
 'B-YEAR-SUBJ': 21,
 'I-DATE-CREATION': 22,
 'I-DATE-PUB': 23,
 'I-DATE-SUBJ': 24,
 'I-GPE': 25,
 'I-GPE-CREATION': 26,
 'I-GPE-DES': 27,
 'I-GPE-PUB': 28,
 'I-GPE-SUBJ': 29,
 'I-I-GPE-DES': 30,
 'I-LITWORK': 31,
 'I-ORG': 32,
 'I-ORG-CREATION': 33,
 'I-ORG-DES': 34,
 'I-ORG-PUB': 35,
 'I-ORG-SUBJ': 36,
 'I-PERSON-AUT': 37,
 'I-PERSON-CON': 38,
 'I-PERSON-EDT': 39,
 'I-PERSON-PUB': 40,
 'I-PERSON-SUBJ': 41,
 'I-YEAR-CREATION': 42,
 'I_ORG-DES': 43,
 'O': 44}

In [32]:
pkl_vars(id2label, 'data/vars/id2label')
pkl_vars(label2id, 'data/vars/label2id')


In [78]:
#splite dataset and load for the first time
"""train_dataset = df.sample(frac=0.9,random_state=888)
valtest_dataset = df.drop(train_dataset.index).reset_index(drop=True)
val_dataset = valtest_dataset.sample(frac=0.5,random_state=888)
test_dataset = valtest_dataset.drop(val_dataset.index).reset_index(drop=True)


train_dataset = train_dataset.reset_index(drop=True)
val_dataset = val_dataset.reset_index(drop=True)
test_dataset = test_dataset.reset_index(drop=True)

#save the data sets
train_dataset.to_csv('data/train.csv', index=False)
val_dataset.to_csv('data/val.csv', index=False)
test_dataset.to_csv('data/test.csv', index=False)"""

In [79]:
"""print("FULL TrainigDataset: {}".format(df.shape))
print("TRAIN Dataset: {}".format(train_dataset.shape))
print("VALIDATION Dataset: {}".format(val_dataset.shape))
print("TEST Dataset: {}".format(test_dataset.shape))"""


FULL TrainigDataset: (1000, 4)
TRAIN Dataset: (900, 4)
VALIDATION Dataset: (50, 4)
TEST Dataset: (50, 4)


In [80]:
"""training_set = dataset(train_dataset, tokenizer, MAX_LEN)
validation_set = dataset(val_dataset, tokenizer, MAX_LEN)
test_set = dataset(test_dataset, tokenizer, MAX_LEN)"""

In [82]:
for tag in ne_labels:
    df[f'has_tag_{tag}'] = df['ner_tags'].apply(lambda x: tag in x)

In [154]:
df = df[~df['has_tag_']]

In [155]:
#5 fold splite dataset and load for the first time

from sklearn.model_selection import KFold

skf = KFold(n_splits=5, random_state=888, shuffle=True)
n = 1                                                        
for trainval_index, test_index in skf.split(X=df['tokens'].to_numpy(), y=df['ner_tags'].to_numpy()):
    trainval_set = df.iloc[trainval_index]
    train_set = trainval_set[:-100]
    val_set = trainval_set[-100:]
    test_set = df.iloc[test_index]
    train_file_name = f'data/5-fold/train_{n}.csv'
    train_set.to_csv(train_file_name, index = False)
    val_file_name = f'data/5-fold/val_{n}.csv'
    val_set.to_csv(val_file_name, index = False)
    test_file_name = f'data/5-fold/test_{n}.csv' 
    test_set.to_csv(test_file_name, index = False)
    n += 1




## verify tokenization


In [22]:
tokenizer(df["sentence"][1])

{'input_ids': [0, 527, 13855, 59134, 152201, 4, 76593, 19, 41505, 745, 86829, 13653, 8402, 3157, 53047, 5581, 64181, 1760, 84838, 647, 4097, 112350, 33, 4, 84838, 13206, 76830, 165, 84838, 429, 1566, 42, 510, 27420, 19, 4, 27420, 19, 67538, 14949, 29036, 4, 84838, 919, 7859, 271, 141, 5, 164034, 9, 1456, 117184, 33, 7510, 7, 5, 148713, 101445, 7, 4, 165, 226741, 7, 224, 58836, 11443, 927, 7372, 39215, 164, 28480, 4, 84838, 814, 6651, 1179, 33, 81937, 7, 165, 4265, 73, 132235, 1755, 1225, 921, 43552, 7, 122, 120279, 429, 1566, 42, 16587, 186, 53, 745, 821, 16591, 7655, 8055, 20756, 756, 88394, 4, 1858, 168, 8892, 13328, 5, 14197, 7418, 7, 72, 75900, 67, 4, 729, 11548, 893, 68, 5094, 1991, 9136, 5486, 1600, 42, 23167, 67, 52334, 18, 1177, 165, 9493, 567, 1177, 53, 19059, 27802, 224, 6, 178490, 12512, 62850, 71493, 14, 491, 33312, 872, 72, 2347, 379, 510, 2], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1

In [116]:
validation_set = dataset(pd.read_csv('data/5-fold/val_1.csv', converters={"tokens":literal_eval, "ner_tags":literal_eval}), tokenizer, MAX_LEN)

In [81]:
# print the first 50 tokens and corresponding labels
for token, label in zip(tokenizer.convert_ids_to_tokens(validation_set[14]["ids"]), validation_set[14]["targets"]):
    print('{0:15}  {1}'.format(token, id2label[label.item()]))


<unk>            O
▁Von             O
▁Gottes          O
▁G               O
na               O
den              O
▁                O
,                O
▁Friedrich       B-PERSON-AUT
▁August          I-PERSON-AUT
▁                O
,                O
▁König           O
▁in              O
▁Po              B-GPE-DES
hlen             B-GPE-DES
▁                O
,                O
▁[               O
et               O
]                O
c                O
.                O
▁Her             O
tz               O
og               O
▁zu              O
▁Sachsen         B-GPE-DES
▁                O
,                O
▁J               B-GPE-DES
ü                B-GPE-DES
lich             B-GPE-DES
▁                O
,                O
▁Cle             B-GPE-DES
ve               B-GPE-DES
▁                O
,                O
▁Berg            B-GPE-DES
▁                O
,                O
▁Eng             B-GPE-DES
ern              B-GPE-DES
▁und             O
▁West            B-GPE-DES
phal    

# Training

In [92]:
"""ids = validation_set[0]["ids"].unsqueeze(0)
mask = validation_set[0]["mask"].unsqueeze(0)
targets = validation_set[0]["targets"].unsqueeze(0)

ids = ids.to(device)#, dtype = torch.long)
mask = mask.to(device)#, dtype = torch.long)
targets = targets.to(device)#, dtype = torch.long)
model.to(device)
outputs = model(input_ids=ids, attention_mask=mask, labels=targets)
initial_loss = outputs[0]

print(f"intial loss = {initial_loss.item()}") """

intial loss = 4.148297309875488


In [72]:
import math

-math.log(1/45) # 45 labels

3.8066624897703196

In [9]:
# Defining the training function on the 80% of the dataset for tuning the bert model
def train(model, training_loader, optimizer, scheduler=None):
    tr_loss, tr_accuracy = 0, 0
    nb_tr_examples, nb_tr_steps = 0, 0
    tr_preds, tr_labels = [], []
    # put model in training mode
    model.train()
    
    for idx, batch in enumerate(training_loader):
        
        ids = batch['ids'].to(device, dtype = torch.long)
        mask = batch['mask'].to(device, dtype = torch.long)
        targets = batch['targets'].to(device, dtype = torch.long)

        outputs = model(input_ids=ids, attention_mask=mask, labels=targets)
        loss, tr_logits = outputs.loss, outputs.logits
        '''
        loss, tr_logits  = model(input_ids=ids, attention_mask=mask, labels=targets)#temporary modification for transformer 3'''
        
        tr_loss += loss.item()

        nb_tr_steps += 1
        nb_tr_examples += targets.size(0)
        
        #if idx % 100==0:
        #    loss_step = tr_loss/nb_tr_steps
        #    print(f"Training loss per 100 training steps: {loss_step}")
           
        # compute training accuracy
        flattened_targets = targets.view(-1) # shape (batch_size * seq_len,)
        active_logits = tr_logits.view(-1, model.num_labels) # shape (batch_size * seq_len, num_labels)
        flattened_predictions = torch.argmax(active_logits, axis=1) # shape (batch_size * seq_len,)
        # now, use mask to determine where we should compare predictions with targets (includes [CLS] and [SEP] token predictions)
        active_accuracy = mask.view(-1) == 1 # active accuracy is also of shape (batch_size * seq_len,)
        targets = torch.masked_select(flattened_targets, active_accuracy)
        predictions = torch.masked_select(flattened_predictions, active_accuracy)
        
        tr_preds.extend(predictions)
        tr_labels.extend(targets)
        
        tmp_tr_accuracy = accuracy_score(targets.cpu().numpy(), predictions.cpu().numpy())
        tr_accuracy += tmp_tr_accuracy
    
        # gradient clipping
        torch.nn.utils.clip_grad_norm_(
            parameters=model.parameters(), max_norm=MAX_GRAD_NORM
        )
        
        # backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        #scheduler.step()

    epoch_loss = tr_loss / nb_tr_steps
    tr_accuracy = tr_accuracy / nb_tr_steps
    #print(f"Trained {nb_tr_steps} steps")
    logger.info(f"Training loss epoch: {epoch_loss}")
    logger.info(f"Training accuracy epoch: {tr_accuracy}")
    

def valid(model, validation_loader):
    # put model in evaluation mode
    model.eval()
    
    eval_loss, eval_accuracy = 0, 0
    nb_eval_examples, nb_eval_steps = 0, 0
    eval_preds, eval_labels = [], []
    
    with torch.no_grad():
        for idx, batch in enumerate(validation_loader):
            
            ids = batch['ids'].to(device, dtype = torch.long)
            mask = batch['mask'].to(device, dtype = torch.long)
            targets = batch['targets'].to(device, dtype = torch.long)
            
           
            outputs = model(input_ids=ids, attention_mask=mask, labels=targets)
            loss, eval_logits = outputs.loss, outputs.logits
            
            eval_loss += loss.item()

            nb_eval_steps += 1
            nb_eval_examples += targets.size(0)
        
            #if idx % 100==0:
            #    loss_step = eval_loss/nb_eval_steps
            #    print(f"Validation loss per 100 evaluation steps: {loss_step}")
              
            # compute evaluation accuracy
            flattened_targets = targets.view(-1) # shape (batch_size * seq_len,)
            active_logits = eval_logits.view(-1, model.num_labels) # shape (batch_size * seq_len, num_labels)
            flattened_predictions = torch.argmax(active_logits, axis=1) # shape (batch_size * seq_len,)
            # now, use mask to determine where we should compare predictions with targets (includes [CLS] and [SEP] token predictions)
            active_accuracy = mask.view(-1) == 1 # active accuracy is also of shape (batch_size * seq_len,)
            targets = torch.masked_select(flattened_targets, active_accuracy)
            predictions = torch.masked_select(flattened_predictions, active_accuracy)
            
            eval_labels.extend(targets)
            eval_preds.extend(predictions)
            
            tmp_eval_accuracy = accuracy_score(targets.cpu().numpy(), predictions.cpu().numpy())
            eval_accuracy += tmp_eval_accuracy
    
    #print(eval_labels)
    #print(eval_preds)

    labels = [id2label[id.item()] for id in eval_labels]
    predictions = [id2label[id.item()] for id in eval_preds]

    #print(labels)
    #print(predictions)
    
    eval_loss = eval_loss / nb_eval_steps
    eval_accuracy = eval_accuracy / nb_eval_steps
    logger.info(f"Validation Loss: {eval_loss}")
    logger.info(f"Validation Accuracy: {eval_accuracy}")

    return labels, predictions

def print_reports_to_csv(test_results, model_name, LEARNING_RATE, EPOCHS, trainset_num, batch_size=TRAIN_BATCH_SIZE, report_type=""):
    test_reports = []
    for res in test_results:
        report = classification_report([res['labels']], [res['predictions']], output_dict=True, zero_division=1)#, scheme=scheme.IOB2, mode='strict')
        flattened_report = {str(k+'_'+v_k) : v_v for k,v in report.items() for v_k, v_v in v.items()  }
        flattened_report['trainset_size'] = res['trainset_size']
        flattened_report['model'] = res['model']
        flattened_report['trainset_num'] = trainset_num
        test_reports.append(flattened_report)
    
    df_test_reports = pd.DataFrame(test_reports)
    if '/' in model_name:
        model_name =  model_name.split('/')[1] 
    test_report_name = f'finetuning_results/{report_type}_{model_name }_set-{trainset_num}_{LEARNING_RATE}_{batch_size}_{EPOCHS}.csv'
    df_test_reports.to_csv(test_report_name, mode='a', header=not os.path.exists(test_report_name),index=False)



In [171]:
validation_set = dataset(pd.read_csv('data/5-fold/val_5.csv', 
                                     converters={"tokens":literal_eval, 
                                                 "ner_tags":literal_eval}),
                           tokenizer, MAX_LEN)
validation_loader = DataLoader(validation_set, **val_params)
val_labels, val_predictions =valid(model, validation_loader)

len(val_labels), len(val_predictions)

(12223, 12223)

In [166]:
print_reports_to_csv([{'trainset_size': trainsetsize, 'model': model_name, 'labels': val_labels, 'predictions': val_predictions}],
                     model_name, LEARNING_RATE, EPOCHS, 2, batch_size=TRAIN_BATCH_SIZE, report_type="val")

In [22]:
%%time
#training
train_params = {'batch_size': TRAIN_BATCH_SIZE,
            'shuffle': False,
            'num_workers': 0
            }

val_params = {'batch_size': VALID_BATCH_SIZE,
                'shuffle': False,
                'num_workers': 0
             }


for trainset_num in range(2,6):

    trainval_file_name = f'data/5-fold/train_{trainset_num}.csv'
    val_file_name = f'data/5-fold/val_{trainset_num}.csv'
    test_file_name = f'data/5-fold/test_{trainset_num}.csv'
    
    for model_name in ["distilbert/distilbert-base-multilingual-cased",
                      "dbmdz/bert-base-german-cased",
                      "deepset/bert-base-german-cased-oldvocab"]:#"dbmdz/bert-base-german-europeana-cased"
        tokenizer = AutoTokenizer.from_pretrained(model_name, from_tf=False, model_max_length=MAX_LEN)
        df_training_set = pd.read_csv(trainval_file_name,converters={"tokens":literal_eval, "ner_tags":literal_eval})        
        validation_set = dataset(pd.read_csv(val_file_name, converters={"tokens":literal_eval, "ner_tags":literal_eval}),
                           tokenizer, MAX_LEN)
        test_set = dataset(pd.read_csv(test_file_name, converters={"tokens":literal_eval, "ner_tags":literal_eval}),
                           tokenizer, MAX_LEN)
        
        val_results = []
        test_results = []
        
        validation_loader = DataLoader(validation_set, **val_params)
        test_loader = DataLoader(test_set, **val_params)
        
        for trainsetsize in [168, 338, 698]:  # till 168, 338, 698
            training_set = dataset(df_training_set[:trainsetsize], tokenizer, MAX_LEN)
        
            logger.info("TRAIN Dataset: {}".format(training_set.data.shape))
            #train_params['batch_size'] =  int( trainsetsize / 32) if (trainsetsize < 1024) else 16
            training_loader = DataLoader(training_set, **train_params)
        
        
            num_training_steps = int(training_loader.dataset.len / train_params['batch_size'] * EPOCHS)
            logger.info(f'tranining steps: {num_training_steps+1}')
        
            #Shrey uses TF model
            model = AutoModelForTokenClassification.from_pretrained(model_name,
                                                                    from_tf=False,
                                                                    num_labels=len(id2label),
                                                                    id2label=id2label,
                                                                    label2id=label2id)
            model.to(device)
        
            optimizer = torch.optim.Adam(params=model.parameters(), lr=LEARNING_RATE)
            #scheduler = get_cosine_schedule_with_warmup(optimizer = optimizer, num_warmup_steps = 50, num_training_steps=num_training_steps)
            for epoch in range(EPOCHS):
            #for epoch in range(flex_epoch_nb): 
                logger.info(f"Training epoch: {epoch + 1}")
                train(model, training_loader, optimizer)
                valid(model, validation_loader)
            val_labels, val_predictions = valid(model, validation_loader)     
            val_results.append({'trainset_size': trainsetsize, 'model': model_name, 'labels': val_labels, 'predictions': val_predictions})
        
            #test generalizablity
            test_labels, test_predictions = valid(model, test_loader)
            test_results.append({'trainset_size': trainsetsize, 'model': model_name, 'labels': test_labels, 'predictions': test_predictions})
            
            ner_model_name = f'ft_models/{model_name.split("/")[-1]}_ft_{EPOCHS}ep_train_size_{trainsetsize}_trainset_{trainset_num}'
            model.save_pretrained(ner_model_name)
            tokenizer.save_pretrained(ner_model_name)
            
            # gpt_aligned_eval(model, tokenizer, ner_model_name) # too slow!
        
        print_reports_to_csv(val_results, model_name, LEARNING_RATE, EPOCHS, trainset_num, report_type='validation')
        print_reports_to_csv(test_results, model_name, LEARNING_RATE, EPOCHS, trainset_num, report_type='test')

/opt/conda/lib/python3.10/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: I_ORG-DES seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/opt/conda/lib/python3.10/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: I_ORG-DES seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/opt/conda/lib/python3.10/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: I_ORG-DES seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))


CPU times: user 56min 59s, sys: 10min 1s, total: 1h 7min 1s
Wall time: 1h 6min 9s


In [12]:
del model